Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=github%2Fbitd-posita-dyce-9%2Fbitd.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/github/bitd-posita-dyce-9)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“BitD Example”](https://github.com/posita/dyce/discussions/9)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H, P
from dyce.evaluation import PResult, PWithSelection, foreach
from enum import IntEnum, auto

class BitdResult(IntEnum):
    BAD_TIMES = auto()
    MESSY_SUCCESS = auto()
    CLEAN_SUCCESS = auto()
    CRITICAL_SUCCESS = auto()

d_bitd = H((
    BitdResult.BAD_TIMES,  # 1
    BitdResult.BAD_TIMES,  # 2
    BitdResult.BAD_TIMES,  # 3
    BitdResult.MESSY_SUCCESS,  # 4
    BitdResult.MESSY_SUCCESS,  # 5
    BitdResult.CLEAN_SUCCESS,  # 6
))

def bitd(n: int) -> H:
    assert n >= 0

    def _mechanic(res: PResult):
        assert len(res.roll) >= 2
        if res.roll[-2] == BitdResult.CLEAN_SUCCESS:
            # Because rolls are ordered least-to-greatest, if the penultimate
            # value is a clean success (i.e., a six), then the ultimate value
            # must be one as well; in other words, we have at least two clean
            # successes (sixes) and therefore a critical success
            return BitdResult.CRITICAL_SUCCESS
        else:
            # Return the highest value, whatever it may be
            return BitdResult(res.roll[-1])

    if n == 0:
        # Take lower of two dice (no critical success possible)
        return (2 @ P(d_bitd)).h(0)
    elif n == 1:
        # Just return a single die (also no critical success possible)
        return d_bitd
    else:
        # We're rolling a pool of dice, so engage our mechanic
        return foreach(_mechanic, PWithSelection(n @ P(d_bitd), (slice(-2, None),)))

%timeit bitd(30)  # works for huge pools

136 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [3]:
from anydyce import jupyter_visualize

results: list[str, H] = []

for n in range(9):
    h = bitd(n)
    h = H((BitdResult(outcome), count) for outcome, count in h.items())  # reconstruct with enum outcomes
    results.append((f"{n}d", h))

jupyter_visualize(results, initial_burst_zero_fill_normalize=True)